In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2

In [2]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓
자켓 해야함 안돌림~~~

In [73]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

# 키워드를 검색한 유튜브 주소 접속
keyword = '블레이저 자켓'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
driver.get(url)

#사이트 최대화
driver.maximize_window()

# 필터 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
time.sleep(1)

# 구분 -> 동영상 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

# 업로드 날짜 -> 올해 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()

In [74]:
# 제목 & 링크 크롤링

# 스크롤 다운 50번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,50):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

print('영상 개수: ', len(name_list))

영상 개수:  58


In [ ]:
#업로드일 가져오기
day_list = [] #업로드일

#업로드 일자 가져오기
for i in range(0,len(name_list)):
    
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i+1, ': 쇼츠', end=' ')

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(2)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(2)

        #업로드일 정보 가져오기
        day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

        print(day)
        day_list.append(day)

    else :
        print(i+1, ': 일반', end=' ')

        driver.get(url_list[i])
            
        time.sleep(2)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(2)  

        #업로드일 정보 가져오기
        day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
        #day = driver.find_element_by_id("info-strings").text.split('\n')[0]
        
        time.sleep(2) 

        print(day)
        day_list.append(day)

In [66]:
day_list2 = [] #전처리 후 업로드일

#업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

#df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
}

youtubeDf = pd.DataFrame(youtubeDic)
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])
youtubeDf.tail()

,제목,업로드일,주소
149,[DIY] 봄맞이 트위드 자켓 만들기🧵｜모노테이프★ #shorts,2022-02-24,https://www.youtube.com/shorts/bfrfruGEss8
150,샤넬 트위드자켓 보이런던백 목걸이 팔찌 언박싱 CHANEL,2021-07-20,https://www.youtube.com/watch?v=agptHTT7V9I
151,트렌디한 트위드 자켓! 셀린 스타일 #Shorts #셀린자켓 #celine,2021-10-21,https://www.youtube.com/shorts/k6HUbjxQR0M
152,일상 브이로그 | 자라 하울을 곁들인 트위드 자켓 소개 | 밀키트로 요리하는 하루 ...,2022-03-16,https://www.youtube.com/watch?v=3XfpHbJjSMk
153,1분안에 트위드자켓 가성비 상품 추천순위 top10 정리,2022-04-21,https://www.youtube.com/watch?v=9p9vkYAE23E


In [59]:
#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url) values (%s, %s, %s);'
    return command

In [67]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = input('테이블명 입력 :')

#table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2]))
    conn.commit()


In [70]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

InterfaceError: cursor already closed

In [69]:
#db 연결 종료
conn.cursor().close()
conn.close()

create table yt_clothes (
	id serial,
	title varchar(100),
	date date,
	url varchar(200));